# Chapter 7: 使用示例 (Few-Shot Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)


## 设置
运行以下设置单元格以加载您的 API 密钥并建立 get_completion 辅助函数。

In [4]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程
给 Claude 提供你希望它如何表现（或不希望它如何表现）的示例非常有效，可以帮助你：
- 得到正确的答案
- 得到正确格式的答案
这种提示方式也被称为 **“少样本提示（few shot prompting）”**。你可能还会遇到“零样本（zero-shot）”、“n-shot”或“一样本（one-shot）”这样的说法。这里的“shot”数量指的是在提示中使用了多少个示例。

## 示例
假设你是一名开发者，正在尝试构建一个能回答孩子问题的“家长机器人”。**LLM默认的回答非常正式和机械**。这会让孩子感到很失望

In [2]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

I don't actually have any information about whether Santa will bring you presents on Christmas. Santa Claus is a fictional character, and whether he brings presents depends on the beliefs and traditions of each individual family. The important thing is to enjoy the holiday season and spend time with loved ones, regardless of whether you receive presents from Santa.


你可以花时间描述你想要的语气，但其实只需**给 Claude 一些理想回复的示例会更简单**。

In [3]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

A: Yes, my dear, as long as you've been good this year, Santa will be sure to bring you presents on Christmas! Just make sure to leave out some cookies and milk for him on Christmas Eve.



在下面的格式化示例中，我们可以一步步地引导LLM 按照一套格式化指令来提取姓名和职业，并按我们想要的方式进行格式化，或者我们也可以直接**给 LLM 一些格式正确的示例**，让 LLM 自行类推。请注意在 assistant 轮次中使用 <individuals>，这样可以让 LLM 从正确的起点开始。

In [6]:
# Prompt template with a placeholder for the variable content
PROMPT = """“银雾谷”是一个迷人的村庄，这里居住着一群非凡的人物。
其中有利亚姆·帕特尔医生，一位神经外科医生，他在地区医疗中心革新了外科手术技术。
奥利维亚·陈是一位创新型建筑师，她用可持续且令人惊叹的设计改变了村庄的风貌。
当地剧院里回荡着伊桑·科瓦奇的迷人交响乐，他是一位受过专业训练的音乐家和作曲家。
伊莎贝拉·托雷斯是一位自学成才的厨师，热衷于使用本地食材，她的农场到餐桌餐厅成为美食爱好者必去的目的地，掀起了一股美食热潮。
这些杰出的人物，各自拥有独特的才华，共同为银雾谷的多彩生活增添了光彩。
<individuals>
利亚姆·帕特尔医生 [神经外科医生]
奥利维亚·陈 [建筑师]
伊桑·科瓦奇 [音乐家和作曲家]
伊莎贝拉·托雷斯 [厨师]
</individuals>
在小镇的中心，主厨奥利弗·汉密尔顿通过他的农场到餐桌餐厅“绿盘”彻底改变了当地的美食格局。奥利弗致力于采购本地有机食材，这让他的餐厅赢得了美食评论家和当地人的一致好评。
就在街道的尽头，你会发现“河畔林地图书馆”，馆长伊丽莎白·陈一直努力为所有人营造一个温馨包容的空间。她扩展了图书馆的服务，并为儿童设立了阅读项目，对小镇的识字率产生了重大影响。
漫步在迷人的小镇广场，你会被墙上美丽的壁画所吸引。这些杰作出自著名艺术家伊莎贝拉·托雷斯之手，她擅长捕捉河畔林地的精髓，为小镇注入了生机。
河畔林地的体育成就同样值得一提，这要归功于前奥运游泳运动员、现任教练马库斯·詹金斯。马库斯利用自己的经验和热情训练小镇的青少年，带领河畔林地游泳队多次获得地区冠军。
<individuals>
奥利弗·汉密尔顿 [厨师]
伊丽莎白·陈 [图书馆馆长]
伊莎贝拉·托雷斯 [艺术家]
马库斯·詹金斯 [教练]
</individuals>
橡树谷是一个迷人的小镇，这里有三位杰出人物，他们的技能和奉献精神对社区产生了深远影响。
在小镇热闹的农贸市场，你会见到劳拉·西蒙斯，一位热情的有机农民，以美味且可持续种植的农产品闻名。她致力于推广健康饮食，激励小镇居民拥抱更环保的生活方式。
在橡树谷社区中心，凯文·阿尔瓦雷斯是一位技艺高超的舞蹈教练，他让各年龄段的人都感受到了舞蹈的乐趣。他包容的舞蹈课程促进了居民间的团结和自我表达，丰富了当地的艺术氛围。
最后，瑞秋·奥康纳是一位不知疲倦的志愿者，投身于各种慈善事业。她致力于改善他人生活，为橡树谷营造了强烈的社区归属感。
凭借他们独特的才华和坚定的奉献精神，劳拉、凯文和瑞秋已经融入橡树谷的生活，帮助小镇变得更加充满活力和繁荣。"""

# Prefill for Claude's response
PREFILL = "<individuals>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
“银雾谷”是一个迷人的村庄，这里居住着一群非凡的人物。
其中有利亚姆·帕特尔医生，一位神经外科医生，他在地区医疗中心革新了外科手术技术。
奥利维亚·陈是一位创新型建筑师，她用可持续且令人惊叹的设计改变了村庄的风貌。
当地剧院里回荡着伊桑·科瓦奇的迷人交响乐，他是一位受过专业训练的音乐家和作曲家。
伊莎贝拉·托雷斯是一位自学成才的厨师，热衷于使用本地食材，她的农场到餐桌餐厅成为美食爱好者必去的目的地，掀起了一股美食热潮。
这些杰出的人物，各自拥有独特的才华，共同为银雾谷的多彩生活增添了光彩。
<individuals>
利亚姆·帕特尔医生 [神经外科医生]
奥利维亚·陈 [建筑师]
伊桑·科瓦奇 [音乐家和作曲家]
伊莎贝拉·托雷斯 [厨师]
</individuals>
在小镇的中心，主厨奥利弗·汉密尔顿通过他的农场到餐桌餐厅“绿盘”彻底改变了当地的美食格局。奥利弗致力于采购本地有机食材，这让他的餐厅赢得了美食评论家和当地人的一致好评。
就在街道的尽头，你会发现“河畔林地图书馆”，馆长伊丽莎白·陈一直努力为所有人营造一个温馨包容的空间。她扩展了图书馆的服务，并为儿童设立了阅读项目，对小镇的识字率产生了重大影响。
漫步在迷人的小镇广场，你会被墙上美丽的壁画所吸引。这些杰作出自著名艺术家伊莎贝拉·托雷斯之手，她擅长捕捉河畔林地的精髓，为小镇注入了生机。
河畔林地的体育成就同样值得一提，这要归功于前奥运游泳运动员、现任教练马库斯·詹金斯。马库斯利用自己的经验和热情训练小镇的青少年，带领河畔林地游泳队多次获得地区冠军。
<individuals>
奥利弗·汉密尔顿 [厨师]
伊丽莎白·陈 [图书馆馆长]
伊莎贝拉·托雷斯 [艺术家]
马库斯·詹金斯 [教练]
</individuals>
橡树谷是一个迷人的小镇，这里有三位杰出人物，他们的技能和奉献精神对社区产生了深远影响。
在小镇热闹的农贸市场，你会见到劳拉·西蒙斯，一位热情的有机农民，以美味且可持续种植的农产品闻名。她致力于推广健康饮食，激励小镇居民拥抱更环保的生活方式。
在橡树谷社区中心，凯文·阿尔瓦雷斯是一位技

---

## 练习
- [Exercise 7.1 - Email Formatting via Examples](#exercise-71---email-formatting-via-examples)

### Exercise 7.1 - 通过示例进行邮件格式化
我们将重新做一次练习 6.2，但这一次，我们要编辑 PROMPT，使用“少样本”示例（即邮件+正确分类和格式化），让 Claude 输出正确的答案。我们希望 Claude 输出的最后一个字母是类别的字母。
如果你忘记每封邮件对应哪个字母分类，请参考 EMAILS 列表中每封邮件旁边的注释。
请记住，这些是邮件的分类：

Remember that these are the categories for the emails:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)								

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """
Please classify emails into the following categories {email}

"""

# Prefill for Claude's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

你需要为 Claude 编写一些示例邮件并进行分类（格式要与您期望的输出完全一致）。有多种方法可以实现。以下是一些指导建议：
1. 尽量至少提供两封示例邮件。Claude 并不需要每个类别都有示例，示例内容也不需要很长。为你认为较难分类的类别提供示例会更有帮助（在第6章练习1结尾处已经让你思考过这个问题）。XML 标签可以帮助你将示例与提示的其他部分分开，虽然这不是必须的。
2. 确保你的示例答案格式与希望 Claude 使用的格式完全一致，这样 Claude 才能模仿这种格式。该格式应确保 Claude 的答案以类别字母结尾。无论你把 {email} 占位符放在哪里，都要确保它的格式与示例邮件完全一致。
3. 确保在提示中仍然列出了所有分类，否则 Claude 不知道要参考哪些分类，同时也要保留 {email} 作为替换用的占位符。

Still stuck? Run the cell below for an example solution.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### 恭喜！
如果您已经完成了到目前为止的所有练习，您就可以进入下一章了。祝您提示工程愉快